In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
_ = """
Any running Python program has many dictionaries active at the same time, even if the user’s program code doesn’t explicitly use a dictionary
Hash tables are the engines behind Python’s high-performance dicts.
We also cover sets in this chapter because they are implemented with hash tables as well.
"""

In [3]:
# generic mapping types
_ = """
An object is hashable if it has a hash value which never changes during its lifetime (it
needs a __hash__() method), and can be compared to other objects (it needs an
__eq__() method). Hashable objects which compare equal must have the same hash
value. […]
the atomic immutable types(str, bytes, numeric types) are all hashable.
A frozen set is always hashable
A tuple is hashable only if all its items are hashable

User-defined types are hashable by default because their hash value is their id() and
they all compare not equal. If an object implements a custom __eq__ that takes into
account its internal state, it may be hashable only if all its attributes are immutable.
"""
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

In [14]:
# dict comprehensions
DIAL_CODES = [
        (86, 'China'),
        (91, 'India'),
        (1, 'United States'),
        (62, 'Indonesia'),
        (55, 'Brazil'),
        (92, 'Pakistan'),
        (880, 'Bangladesh'),
        (234, 'Nigeria'),
        (7, 'Russia'),
        (81, 'Japan'),
    ]
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [15]:
_ = """
The way update handles its first argument m is a prime example of duck typing: it first
checks whether m has a keys method and, if it does, assumes it is a mapping. Otherwise,
update falls back to iterating over m, assuming its items are (key, value) pairs. The
constructor for most Python mappings uses the logic of update internally, which means
they can be initialized from other mappings or from any iterable object producing (key,
value) pairs.
"""
country_code
country_code.values()
country_code.popitem()

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

dict_values([86, 91, 1, 62, 55, 92, 880, 234, 7, 81])

('Japan', 81)

In [16]:
# handling missing keys with setdefault
d = {'r': 1, 'o': 2}
d.get('oo', 0)
d.setdefault('oo', 2)
d.get('oo', 0)

0

2

2

In [9]:
# mappings with flexible key lookup
_ = """
Sometimes it is convenient to have mappings that return some made-up value when a
missing key is searched. There are two main approaches to this: one is to use a default
dict instead of a plain dict. The other is to subclass dict or any other mapping type
and add a __missing__ method. Both solutions are covered next
"""
# defaultdict
_ = """
Here is how it works: when instantiating a defaultdict, you provide a callable that is
used to produce a default value whenever __getitem__ is passed a nonexistent key
argument.

when dd = defaultdict(list)
1. Calls list() to create a new list.
2. Inserts the list into dd using 'new-key' as key.
3. Returns a reference to that list.

The callable that produces the default values is held in an instance attribute called
default_factory and is only invoked to provide default values for __getitem__ calls
dd[k] will call but dd.get(k) will not
"""
from collections import defaultdict

dd = defaultdict(list)
dd[2]
dd.get(3)

[]

In [ ]:
# the __missing__ method
_ = """
Underlying the way mappings deal with missing keys is the aptly named __missing__
method. This method is not defined in the base dict class, but dict is aware of it: if you
subclass dict and provide a __missing__ method, the standard dict.__getitem__ will
call it whenever a key is not found, instead of raising KeyError.

The __missing__ method is just called by __getitem__ (i.e., for
the d[k] operator). The presence of a __missing__ method has no
effect on the behavior of other methods that look up keys, such as
get or __contains__ (which implements the in operator). This is
why the default_factory of defaultdict works only with
__getitem__, as noted in the warning at the end of the previous
section.
"""
# when searching for a nonstring key, StrKeyDict0 converts it to str when it is not found
class StrKeyDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str):    # very necessary
            raise KeyError(key)
        return self[str(key)]

    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default

    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()


d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']
d[4]
d.get('2')
d.get(4)
2 in d
1 in d

_ = """
A search like k in my_dict.keys() is efficient in Python 3 even
for very large mappings because dict.keys() returns a view,
which is similar to a set, and containment checks in sets are as
fast as in dictionaries. Details are documented in the “Dictionary”
view objects section of the documentation. In Python 2,
dict.keys() returns a list, so our solution also works there, but
it is not efficient for large dictionaries, because k in my_list
must scan the list.
"""

In [3]:
# variations of dict
import collections

_ = """
collections.OrderedDict
    Maintains keys in insertion order, allowing iteration over items in a predictable
    order. The popitem method of an OrderedDict pops the first item by default, but
    if called as my_odict.popitem(last=True), it pops the last item added.

collections.ChainMap
    Holds a list of mappings that can be searched as one. The lookup is performed on
    each mapping in order, and succeeds if the key is found in any of them. This is useful
    to interpreters for languages with nested scopes, where each mapping represents a
    scope context. The “ChainMap objects” section of the collections docs has several
    examples of ChainMap usage, including this snippet inspired by the basic rules of
    variable lookup in Python:
    
collections.Counter
    A mapping that holds an integer count for each key. Updating an existing key adds
    to its count. This can be used to count instances of hashable objects (the keys) or
    as a multiset—a set that can hold several occurrences of each element. Counter
    implements the + and - operators to combine tallies, and other useful methods such
    as most_common([n]), which returns an ordered list of tuples with the n most common
    items and their counts; see the documentation. Here is Counter used to count
    letters in words:
"""
ct = collections.Counter('abracadabra')
ct
ct.update('robin')
ct
ct.most_common(2)


Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

Counter({'a': 5, 'b': 3, 'r': 3, 'c': 1, 'd': 1, 'o': 1, 'i': 1, 'n': 1})

[('a', 5), ('b', 3)]

In [10]:
# subclassing UserDict
_ = """
collections.UserDict
    A pure Python implementation of a mapping that works like a standard dict

The main reason why it’s preferable to subclass from UserDict rather than from dict
is that the built-in has some implementation shortcuts that end up forcing us to override
methods that we can just inherit from UserDict with no problems

Note that UserDict does not inherit from dict, but has an internal dict instance, called data, which holds the actual items
"""
class StrKeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):  # very necessary
            raise KeyError(key)
        return self[str(key)]

    def __setitem__(self, key, item):
        self.data[str(key)] = item

    def __contains__(self, key):
        return str(key) in self.data


In [12]:
# immutable mappings
_ = """
Since Python 3.3, the types module provides a wrapper class called MappingProxy
Type, which, given a mapping, returns a mappingproxy instance that is a read-only but
dynamic view of the original mapping. This means that updates to the original mapping
can be seen in the mappingproxy, but changes cannot be made through it. See
Example 3-9 for a brief demonstration.
"""
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy
d_proxy[1]
d[2] = 'B'
d_proxy
d_proxy[2] = 'x'


mappingproxy({1: 'A'})

'A'

mappingproxy({1: 'A', 2: 'B'})

TypeError: 'mappingproxy' object does not support item assignment

In [15]:
# set theory
_ = """
frozenset is immutable addition of set
A set is a collections of unique objects
"""
l = ['spam', 'spam', 'eggs', 'spam']
set(l)
list(set(l))
_ = """
set elements must be hashable, set type is not hashable but frozenset is, so you can have frozenset inside a set

a | b: union
a & b: intersection
a - b: difference

"""
# Count occurrences of needles in a haystack
# found = len(needles & haystack)
# found = len(set(needles) & set(haystack))
# found = len(set(needles).intersection(haystack))

{'eggs', 'spam'}

['spam', 'eggs']

In [20]:
# set literals
# no literal notation for the empty set, write set()!, {} is to create a empty dictionary
s = {1}
type(s)
s
s.pop()
s

_ = """
Literal set syntax like {1, 2, 3} is both faster and more readable than calling the
constructor (e.g., set([1, 2, 3])). The latter form is slower because, to evaluate it,
Python has to look up the set name to fetch the constructor, then build a list, and finally
pass it to the constructor. In contrast, to process a literal like {1, 2, 3}, Python runs
a specialized BUILD_SET bytecode.
"""
from dis import dis  # disassembler function to see bytecodes
dis('{1}')

set

{1}

1

set()

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [18]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


In [19]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

In [21]:
# set comprehensions
from unicodedata import name
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

In [2]:
# Practical Consequences of How dict Works
# dicts have significant memory overhead
_ = """
because a dict uses a hash table internally, and hash table must be sparse to work, they are not memory efficient
For example, if you are handling a large quantity of records, it
makes sense to store them in a list of tuples or named tuples instead of using a list of
dictionaries in JSON style, with one dict per record. Replacing dicts with tuples reduces
the memory usage in two ways: by removing the overhead of one hash table per record
and by not storing the field names again with each record.

key search is very fast
The dict implementation is an example of trading space for time: dictionaries have
significant memory overhead, but they provide fast access regardless of the size of the
dictionary

key ordering depends on insertion order
d1 = {x1, x2} == d2 = {x2, x1}

adding items to a dict may change the order of existing keys
Whenever you add a new item to a dict, the Python interpreter may decide that the
hash table of that dictionary needs to grow. This entails building a new, bigger hash
table, and adding all current items to the new table.

do not modify dict while scanning
.keys(), .items(), .values() return dictionary views, which behave more like sets than the lists

当占用空间>1/3后 会自动扩容
"""


In [4]:
# how sets work——practical consequences
_ = """
the set and frozenset types are also implemented with a hash table
"""